In [1]:
import networkx as nx         #pacchetto per reti
import numpy as np
import ipyparallel as ipp     #pacchetto per parallelizzazione
import time as clock
import matplotlib.pyplot as plt
from scipy import stats
from scipy import optimize
import pickle
#import winsound
from Node import *            #pacchetto agente Node
from NetworkTool import *     #pacchetto rete
from GenerativeTool import *  #pacchetto generazione rete
from VoteTool import *        #pacchetto analisi voto
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def Core(MG, Nodes, NodesA, FB, VB, RndFr, fbpos, fbdist,vbpos, vbdist, LayerA, LayerC, TheTime):
     
    for i in range(Nodes):
        tfix=list(-2*np.random.random(FB)+1)
        tvar=list(-2*np.random.random(VB)+1)
        MG.add_node(Node(MG.number_of_nodes(),tfix,tvar, np.random.random(),TheTime)) #create nodes with FB fix belief and VB var beli
        
    TheTime=TheTime+1

    for i in range(MG.number_of_nodes()):
        tlay = np.random.randint(LayerC)
        MG.nodes()[i].aggF(TheTime)
        AttachFriendFirst(MG, MG.nodes()[i], np.random.randint(RndFr)+1, tlay, fbpos[tlay], fbdist[tlay], vbpos[tlay], vbdist[tlay],TheTime) #every node search for friends
    print(' Time ', TheTime)

In [3]:
def Grow(MG, Nodes, NodesA, FB, VB, RndFr, fbpos, fbdist,vbpos, vbdist, LayerA, LayerC, TheTime):
    for j in range(Iter): #creating and adding
        TheTime = TheTime+1
        LayerB = ((LayerA-LayerC)//(Iter-1))*j + LayerC
        NodesB = NodesA//Iter
        print('Iter ',j,' Layers ', LayerB, ' Time ', TheTime)
        for i in range(NodesB):
            tfix=list(-2*np.random.random(FB)+1)
            tvar=list(-2*np.random.random(VB)+1)
            tnew = Node(MG.number_of_nodes(), tfix, tvar, np.random.random(), TheTime) #create nodes with FB fix belief and VB var belief
            MG.add_node(tnew) 
            tnew.aggF(TheTime)
            tlay = np.random.randint(LayerB)
            AttachChoosenBelief(MG, tnew, np.random.randint(RndFr)+1, tlay, fbpos[tlay], fbdist[tlay], vbpos[tlay], vbdist[tlay], TheTime)

    print(MG.number_of_nodes())
    print(' Time ', TheTime)

In [ ]:
def Rand(MG, Nodes, NodesA, FB, VB, RndFr, fbpos, fbdist,vbpos, vbdist, LayerA, LayerC, TheTime):
    TheTime=TheTime+1 
    co=0
    while MG.number_of_nodes()<NodesA:
        tfix=list(-2*np.random.random(FB)+1)
        tvar=list(-2*np.random.random(VB)+1)
        tnew = Node(MG.number_of_nodes(), tfix, tvar, np.random.random(), TheTime) #create nodes with FB fix belief and VB var belief
        MG.add_node(tnew) 
        tnew.aggF(TheTime)
        
    for i in MG.nodes():
        if len(MG.neighbors(i))==0:
            co=co+1
            RandomLink(MG, i, LayerA, TheTime)
    print("added: ",co,'; percentage: ', (co/MG.number_of_nodes())*100,'% ')
    print(' Time ', TheTime)
    return(co/MG.number_of_nodes())

In [ ]:
AvgDistD = {}
StdDistD = {}
AvgDistC = {}
StdDistC = {}

NodeList=[500,1000,1500,2000]
FBlist=[5]
VBlist=[10]
BDList=[0.02,0.05,0.08]
LayerList=[4,8,12]
InfluDist=[0.2]

AvgDistC['proj'] = {k:[] for k in NodeList+BDList+FBlist+VBlist+LayerList+InfluDist}
StdDistC['proj'] = {k:[] for k in NodeList+BDList+FBlist+VBlist+LayerList+InfluDist}
AvgDistC['alay'] = {k:[] for k in NodeList+BDList+FBlist+VBlist+LayerList+InfluDist}
StdDistC['alay'] = {k:[] for k in NodeList+BDList+FBlist+VBlist+LayerList+InfluDist}
AvgDistD['proj'] = {k:[] for k in NodeList+BDList+FBlist+VBlist+LayerList+InfluDist}
StdDistD['proj'] = {k:[] for k in NodeList+BDList+FBlist+VBlist+LayerList+InfluDist}
AvgDistD['alay'] = {k:[] for k in NodeList+BDList+FBlist+VBlist+LayerList+InfluDist}
StdDistD['alay'] = {k:[] for k in NodeList+BDList+FBlist+VBlist+LayerList+InfluDist}

print('\n', AvgDistC,'\n',StdDistC, '\n', AvgDistD,'\n',StdDistD)
    
for rep in range(2):
    print('\n Repetition',rep)
    for NodesA in NodeList:    
        for bd in BDList:
            for FB in FBlist:
                for VB in VBlist:
                    for LayerA in LayerList:
                        for InDi in InfluDist:

                            #NodesA=random.randint(500,2500)
                            #yeah
                            print('\n Setup',NodesA,FB,VB,LayerA,InDi,bd)

                            MG = nx.MultiGraph()
                            TheTime = 0
                            RndFr=3
                            Nodes=50 #core set of 50 nodes
                            Iter=3
                            LayerC=2 #core layers
                            issue=np.random.random(FB+VB) #voting issue
                            Keys = [FB, VB, LayerA, InDi, bd, NodesA]
                            print(' Time ', TheTime)


                            fbdist, fbpos, vbdist, vbpos, cbpos = [], [], [], [], []
                            for j in range(LayerA): #creates random vector in specified range
                                fbpos.append(list(np.random.randint(FB, size=np.random.randint(1,FB))))
                                fbdist.append(0.50*np.random.random()+0.2)
                            for j in range(LayerA): #creates random vector in specified range
                                vbpos.append(list(np.random.randint(VB, size=np.random.randint(1,VB))))
                                vbdist.append(0.50*np.random.random()+0.2)
                                cbpos.append(list(np.random.randint(VB, size=np.random.randint(1,VB))))
                            print('\n',fbdist,'\n',fbpos,'\n',vbdist,'\n',vbpos,'\n',cbpos)

                            Core(MG, Nodes, NodesA, FB, VB, RndFr, fbpos, fbdist,vbpos, vbdist, LayerA, LayerC, TheTime)
                            Grow(MG, Nodes, NodesA, FB, VB, RndFr, fbpos, fbdist,vbpos, vbdist, LayerA, LayerC, TheTime)
                            print(Rand(MG, Nodes, NodesA, FB, VB, RndFr, fbpos, fbdist,vbpos, vbdist, LayerA, LayerC, TheTime))

                            print(TheTime)

                            #AllVoteA(MG, issue)

                            #BaseAnalisys(MG,LayerA)

                            EvolveInfluR(MG, 50000, cbpos, 0.75, InDi, TheTime, LayerA)
                            print(' Time ', TheTime)

                            AllVoteA(MG, issue)

                            voteMG=[]
                            tmpAvg, tmpStd = [], []
                            for n in MG.nodes():
                                voteMG.append(n.getVote())
                            tmpAvg.append(np.nanmean(voteMG))
                            tmpStd.append(np.nanstd(voteMG))    
                            
                            for layer in range(LayerA):
                                G = ExtractLayer(MG, layer)
                                tt=[]
                                for n in MG.nodes():
                                    tt.append(n.getVote())
                                tmpAvg.append(np.nanmean(tt))
                                tmpStd.append(np.nanstd(tt))
                                
                            for n in MG.nodes():#---------------------------
                                ConformUncertVote(n, MG, bd, issue)
                            
                            t0=[]
                            for n in MG.nodes():
                                t0.append(n.getVote())
                                
                            t1, t2 = [], []
                            for layer in range(LayerA):
                                G = ExtractLayer(MG, layer)
                                tt=[]
                                for n in MG.nodes():
                                    tt.append(n.getVote())
                                t1.append(np.nanmean(tt))
                                t2.append(np.nanstd(tt)) 
                                
                            for key in Keys:
                                AvgDistC['proj'][key].append(np.nanmean(t0)-tmpAvg[0])
                                StdDistC['proj'][key].append(np.nanstd(t0)-tmpStd[0])
                                AvgDistC['alay'][key].append(np.nanmean(t1)-np.nanmean(tmpAvg[1:]))
                                StdDistC['alay'][key].append(np.nanmean(t2)-np.nanmean(tmpStd[1:]))
                                
                            
                            AllVoteA(MG, issue)
                            
                            for n in MG.nodes():#-------------------------------
                                DifformUncertVote(n, MG, bd, issue)    
                           
                            t0=[]
                            for n in MG.nodes():
                                t0.append(n.getVote())
                                
                            t1, t2 = [], []
                            for layer in range(LayerA):
                                G = ExtractLayer(MG, layer)
                                tt=[]
                                for n in MG.nodes():
                                    tt.append(n.getVote())
                                t1.append(np.nanmean(tt))
                                t2.append(np.nanstd(tt)) 
                                
                            for key in Keys:
                                AvgDistD['proj'][key].append(np.nanmean(t0)-tmpAvg[0])
                                StdDistD['proj'][key].append(np.nanstd(t0)-tmpStd[0])
                                AvgDistD['alay'][key].append(np.nanmean(t1)-np.nanmean(tmpAvg[1:]))
                                StdDistD['alay'][key].append(np.nanmean(t2)-np.nanmean(tmpStd[1:])) 
                                


                            MG.clear()
                            print('\n',AvgDistC,'\n',StdDistC,'\n------------------------------------------')
                            print('\n',AvgDistD,'\n',StdDistD,'\n------------------------------------------')
                       
                        
    with open('CDEADC.pickle', 'wb') as handle:
        pickle.dump(AvgDistC, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('CDESDC.pickle', 'wb') as handle:
        pickle.dump(StdDistC, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('CDEADD.pickle', 'wb') as handle:
        pickle.dump(AvgDistD, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('CDESDD.pickle', 'wb') as handle:
        pickle.dump(StdDistD, handle, protocol=pickle.HIGHEST_PROTOCOL)



 {'alay': {0.05: [], 0.08: [], 4: [], 5: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 0.02: [], 0.2: [], 1500: []}, 'proj': {0.05: [], 0.08: [], 4: [], 5: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 0.02: [], 0.2: [], 1500: []}} 
 {'alay': {0.05: [], 0.08: [], 4: [], 5: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 0.02: [], 0.2: [], 1500: []}, 'proj': {0.05: [], 0.08: [], 4: [], 5: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 0.02: [], 0.2: [], 1500: []}} 
 {'alay': {0.05: [], 0.08: [], 4: [], 5: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 0.02: [], 0.2: [], 1500: []}, 'proj': {0.05: [], 0.08: [], 4: [], 5: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 0.02: [], 0.2: [], 1500: []}} 
 {'alay': {0.05: [], 0.08: [], 4: [], 5: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 0.02: [], 0.2: [], 1500: []}, 'proj': {0.05: [], 0.08: [], 4: [], 5: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 0.02: [], 0.2: [],

In [ ]:
print('\n',AvgDistC)
print('\n', StdDistC)
print('\n',AvgDistD)
print('\n', StdDistD)

In [ ]:
#VoteAnalysis(MG, issue, bd,1)
#VoteALayerLight(MG, issue, bd, 0, LayerA)

In [ ]:
#PollLayers(MG, 100, 100, bd, issue, 1, 2, 1)

In [ ]:
#PlotLayer(MG,LayerA)

In [ ]:
funname = ['AvgDistD','StdDistD','AvgDistC','StdDistC']
dictionary = [AvgDistD, StdDistD, AvgDistC, StdDistC]
#n=0
setting = ['alay','proj']
#m=1
varname = ['abstentionBound','Layers', 'Nodes']
varlist = [BDList, LayerList, NodeList]
#o=1

for n in range(len(dictionary)):
    for m in range(len(setting)):
        for o in range(len(varlist)):

            temp0 = []
            temp1 = []
            temp2 = []
            temp3 = []
            print('\n', funname[n], setting[m], varname[o],'\n')

            for j in varlist[o]:
                #print(j)
                temp0.append(dictionary[n][setting[m]][j])
                temp1.append(np.nanmean(dictionary[n][setting[m]][j]))
                temp2.append(np.nanmax(dictionary[n][setting[m]][j]))
                temp3.append(np.nanmin(dictionary[n][setting[m]][j]))

                print('value',j,', average',np.nanmean(dictionary[n][setting[m]][j]),', st dev',np.nanstd(dictionary[n][setting[m]][j]))

            plt.plot(varlist[o],temp0, 'kx')
            plt.plot(varlist[o],temp1, 'r-')
            plt.plot(varlist[o],temp2, 'g-')
            plt.plot(varlist[o],temp3, 'g-')
            plt.ylabel(funname[n])
            plt.xlabel(varname[o])
            plt.savefig('CDE'+funname[n]+setting[m]+varname[o]+'.png',orientation='landscape',bbox_inches='tight',dpi='figure')
            plt.show()


In [ ]:
with open('CDEADC.pickle', 'rb') as handle:
    AvgDistC=pickle.load(handle)
with open('CDESDC.pickle', 'rb') as handle:
    StdDistC=pickle.load(handle)
with open('CDEADD.pickle', 'rb') as handle:
    AvgDistD=pickle.load(handle)
with open('CDESDD.pickle', 'rb') as handle:
    StdDistD=pickle.load(handle)
